## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


확률적 경사 하강법(SGD), 미니배치 경사 하강법

-> 한번에 모든 데이터를 사용하지 않기 때문에

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

과대적합이 일어나고 있다.

-> 해결 : 학습률 낮추기,훈련 멈추기

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

두 오차가 모두 높다 -> 과소적합 -> 높은 편향이 문제 -> 알파 값을 낮춰야 한다.

(알파값이 높을수록 더 강력한 규제가 적용)

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

1. 성능 면에서 규제가 약간 있는 것이 대부분의 경우에 좋다.
2. 쓰이는 특성이 몇 개 뿐이라고 의심되는 경우에 라쏘 회귀가 더 좋다.
3. 특성 수가 훈련 샘플 수 보다 많거나 특성 몇 개가 강하게 연관되어 있을 때 라쏘가 문제를 일으키므로 엘라스틱넷을 사용한다.

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from sklearn.datasets import load_iris
iris = load_iris()

In [121]:
X = iris['data'][:, 3:] # 꽃잎의 너비
y = (iris['target']).astype(int)

In [122]:
# 편향 추가 (x0 = 1)
X = np.c_[np.ones([len(X), 1]), X]

In [123]:
# train/test/validation split
train_ratio = 0.6
test_ratio = 0.2
val_ratio = 0.2

indices = np.random.permutation(len(X)) # 0부터 len(X)까지의 숫자를 임의의 순서로 섞은 배열을 생성

train_size = int(train_ratio * len(X))
test_size = int(test_ratio * len(X))
val_size = len(X) - train_size - test_size

X_train = X[indices[:train_size]]
X_test = X[indices[train_size:train_size + test_size]]
X_val = X[indices[train_size + test_size:]]

y_train = y[indices[:train_size]]
y_test = y[indices[train_size:train_size + test_size]]
y_val = y[indices[train_size + test_size:]]

In [124]:
# 원핫인코딩 함수 정의
def one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    one_hot_y = np.zeros((m, n_classes))
    one_hot_y[np.arange(m), y] = 1
    return one_hot_y

In [125]:
y_train_one_hot = one_hot(y_train)
y_val_one_hot = one_hot(y_val)
y_test_one_hot = one_hot(y_test)

In [126]:
# softmax 함수 정의
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [127]:
# 입/출력 개수 정의
n_inputs = X_train.shape[1] # 2
n_outputs = len(np.unique(y_train))   # == 3 (3개의 붓꽃 클래스)

In [128]:
n_inputs

2

In [138]:
lr = 0.05
n = 5001 # 반복 횟수
m = len(X_train) # 샘플 수
epsilon = 1e-7 # 로그 함수의 값이 0이 되는 것을 방지하기 위한 아주 작은 값
best_loss = np.infty

Theta = np.random.randn(n_inputs, n_outputs) # 가중치 행렬 -> 입력 피처와 출력 클래스 간의 관계를 나타낸다.

for iteration in range(n):
    logits = X_train.dot(Theta) # 각 클래스에 대한 점수
    y_proba = softmax(logits) # 소프트맥스 : 로짓 -> 확률
    if iteration % 500 == 0:
        loss = -np.mean(np.sum(y_train_one_hot * np.log(y_proba + epsilon), axis=1))
        print(iteration, loss)
    error = y_proba - y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - lr * gradients # 파라미터 업데이트

0 2.284134622685217
500 0.621889213976295
1000 0.47427012933732127
1500 0.4078460355312617
2000 0.36605790958463436
2500 0.3363410471535851
3000 0.31385438727370063
3500 0.2961663056473354
4000 0.2818657502075406
4500 0.27005921351720047
5000 0.2601470072467537


In [135]:
Theta # 모델 파라미터 확인

array([[ 5.41199098,  1.42803057, -5.59659277],
       [-5.88056225, -0.05044581,  4.28896622]])

In [136]:
logits = X_val.dot(Theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_val)
accuracy_score

0.9666666666666667

In [137]:
y_predict

array([0, 2, 0, 2, 2, 0, 2, 2, 1, 0, 2, 1, 0, 2, 1, 1, 2, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1])

편향을 추가하지 않으면 class 2로만 예측한다.

In [143]:
# 조기 종료 코드
#     if loss < best_loss:
#         best_loss = loss
#     else:
#         print(iteration - 1, best_loss)
#         print(iteration, loss, "조기 종료!")
#         break